Importing libraries

In [1]:
import pyterrier as pt
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from pyterrier.measures import *
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from pandarallel import pandarallel #to parallelize operations on dataframe for efficiency improving
import fasttext
import spacy
nlp=spacy.load('en_core_web_sm')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from itertools import combinations
import tensorflow as tf
from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import pickle
from tensorflow.keras.models import load_model

In [2]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U git+https://github.com/huggingface/transformers.git
!pip3 install -q -U git+https://github.com/huggingface/peft.git
!pip3 install -q -U git+https://github.com/huggingface/accelerate.git
!pip3 install -q -U einops
!pip3 install -q -U safetensors

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [5]:
# A version of falcon with smaller chunks on safetensors for low RAM environments

################################################# Model 1 -- Performs well
#model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"

################################################# Model 2 - Low performance (?)
#model_id = 'ethzanalytics/RedPajama-INCITE-Instruct-7B-v0.1-sharded-bf16'

################################################# Model 3 --
model_id = 'TinyPixel/Llama-2-7B-bf16-sharded'

################################################### RUN IF you have a COLAB PRO - Session crash due to RAM
################################################## Model 1
#model_id = "bigscience/bloom-7b1"

################################################## Model 2
#!pip install flash-attn --no-build-isolation
#!pip install git+https://github.com/HazyResearch/flash-attention.git#subdirectory=csrc/rotary
#model_id = "togethercomputer/Llama-2-7B-32K-Instruct"

In [7]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [10]:
import torch
import bitsandbytes as bnb

x = torch.randn(10, 10).cuda()
y = bnb.optim.Adam8bit(x, lr=0.001)
print(y)



AssertionError: Torch not compiled with CUDA enabled

In [8]:
import accelerate
import bitsandbytes as bnb
print(accelerate.__version__)
print(bnb.__version__)


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
0.27.0.dev0
0.42.0


/opt/homebrew/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline

In [6]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [1]:
import warnings
warnings.filterwarnings("ignore")